In [0]:
import pandas as pd
from sklearn import tree
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [0]:
dados_treino = pd.read_csv('treino.csv', index_col='Id')
dados_teste = pd.read_csv('teste.csv', index_col='Id')

In [0]:
dados_treino.head()

In [0]:
dados_treino.Alley.isnull()

In [0]:
dados_treino.Alley.isna()

In [0]:
dados_treino.Alley.isna().sum()

In [0]:
dados_treino.Alley.isnull().sum()

In [0]:
dados_treino_copia = dados_treino.copy()

In [0]:
dados_treino_copia.drop('Alley', axis=1).head(5)

In [0]:
X = dados_treino_copia.select_dtypes(exclude=['object'])
X.info()

In [0]:
X.dropna(axis=0, subset=['SalePrice'], inplace=True)

In [0]:
y = X.SalePrice

In [0]:
X.drop(['SalePrice'], axis=1, inplace=True)

In [0]:
X_treino, X_valid, y_treino, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42)


In [0]:
cols_cat = [coluna for coluna in X_treino.columns
            if X_treino[coluna].nunique() < 10 
            and X_treino[coluna].dtype == "object"]

In [0]:
cols_num = [coluna for coluna in X_treino.columns 
            if X_treino[coluna].dtype in ['int64' , 'float64']]

In [0]:
cols_num[:5]

In [0]:
cols = cols_cat + cols_num
X_treino_sel = X_treino[cols].copy()
X_valid_sel = X_valid[cols].copy()
X_teste_sel = dados_teste[cols].copy()


In [0]:
from sklearn.impute import SimpleImputer

In [0]:
transformer_num = SimpleImputer(strategy='median')

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [0]:
transformer_cat = Pipeline(
                    steps=[( 'imputacao', SimpleImputer(strategy='constant')),
                           ('encoding', OneHotEncoder(handle_unknown='ignore'))])

In [0]:
transformers = [('transformer_num', transformer_num, cols_num),
                ('transformer_cat', transformer_cat, cols_cat)]

In [0]:
from sklearn.compose import ColumnTransformer

In [0]:
preprocessor = ColumnTransformer(transformers)

In [0]:
model = RandomForestRegressor(n_estimators=50, random_state=42)
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('model', model) ])

In [0]:
from sklearn.model_selection import cross_val_score
maes = -1 * cross_val_score(pipe, X, y, cv=5, scoring='neg_mean_absolute_error')
maes

array([18249.04086758, 17732.82296804, 18433.61568493, 16276.22940639,
       19856.19047945])

In [0]:
pipe.fit(X_treino_sel, y_treino)

In [0]:
preds = pipe.predict(X_valid_sel)

In [0]:
mae = mean_absolute_error(y_valid, preds)
mae

18400.045182648402